In [129]:
import csv
import os
import glob
import pandas as pd
import re
import nltk
import webbrowser
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

[nltk_data] Downloading package stopwords to /home/ruben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [109]:
def Clean(string):
    strip_special_chars = re.compile("[^A-Za-z]")
    string = re.sub(strip_special_chars," ", string)
    string = string.split()
    string = [w.lower() for w in string]
    string = [word for word in string if len(word) > 2]
    #string = [word for word in string if word not in stopwords.words("dutch")]
    return(string)

In [110]:
os.chdir('/home/ruben/Downloads')

In [111]:
csv_filenames = glob.glob('*.csv')

In [138]:
df = pd.DataFrame()

for fn in csv_filenames:
    tmp = pd.read_csv(fn, sep = "\t")
    df = df.append(tmp)


In [144]:
list_tokens = list()

for i in range(0,70979):
    if "buitenland" in df.ocr[i]:
        token_list = Clean(df.ocr[i])
        list_tokens.append(token_list)
    

KeyError: 47353

In [124]:
list_bigrams = list()

for i in range(0,len(list_tokens)):
    if "buitenland" in list_tokens[i]:
        pos = list_tokens[i].index("buitenland")
        if pos + 5 > len(list_tokens[i]):
            pos = len(list_tokens[i])
        if pos - 5 < 5:
            pos = 0
        
        tmp = " ".join(list_tokens[i][(pos):(pos+2)])
        list_bigrams.append(tmp)

In [136]:
lb = dict(Counter(list_bigrams))
lb

{'': 7,
 'buitenland aan': 2,
 'buitenland aangebouwde': 1,
 'buitenland aangegane': 1,
 'buitenland aangekochte': 1,
 'buitenland aangekomen': 1,
 'buitenland aangeraden': 1,
 'buitenland aangevoerd': 2,
 'buitenland aanhangig': 1,
 'buitenland aanhoudend': 1,
 'buitenland aankomen': 1,
 'buitenland aanraking': 1,
 'buitenland aanvaarden': 1,
 'buitenland achting': 1,
 'buitenland afgesneden': 1,
 'buitenland ainnemen': 1,
 'buitenland algemeene': 1,
 'buitenland alhier': 1,
 'buitenland alle': 2,
 'buitenland alleen': 1,
 'buitenland als': 4,
 'buitenland andere': 1,
 'buitenland baar': 1,
 'buitenland begeven': 2,
 'buitenland behandelen': 1,
 'buitenland behoefde': 1,
 'buitenland behoeven': 4,
 'buitenland bekend': 1,
 'buitenland belangrij': 1,
 'buitenland belemmerenden': 1,
 'buitenland bemoeijelijken': 1,
 'buitenland beraadslagingen': 1,
 'buitenland berigten': 1,
 'buitenland beschouwd': 1,
 'buitenland bespoedigen': 1,
 'buitenland bestaat': 1,
 'buitenland bestemd': 1,
 'b

In [137]:
pd.DataFrame(lb.items())

ValueError: DataFrame constructor not properly called!

In [143]:
len(df)

70980

'willed bij'